<a href="https://colab.research.google.com/github/jethompson023/Kettering-Senior-Thesis-Project/blob/main/Thesis_DataClean_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: For version two look to implement both/or one Polars API or the Numba API

In [ ]:
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
#Imports for our Script
import pandas as pd
import numpy as np
import os
import json
from datetime import *

##Importing of the Data

In [ ]:
#Add google authentication 
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#Import the data from the Google Sheet
workbook = gc.open_by_key('1mjE530k7fYKkPh9VPcqfElrrzMkw59uUTPQ34VWH0wA')
worksheet = workbook.worksheet('POU')

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
pou_data = pd.DataFrame.from_records(rows)

In [ ]:
#Next we are going to format the rows to allow for the data to get into a data frame
pou_data  = pou_data.drop(labels=0, axis=0)
pou_data  = pou_data.reset_index(drop=True)
pou_data.columns = pou_data.iloc[0]
pou_data  = pou_data.iloc[1:].reset_index(drop=True)
pou_data 

,Item_no,Item Description,Qty,Status,Serial Number,Reason Code,Shortage Notes,Team Found,Team Resp,Date Found,Time Found (Pacific),SHORT_FOUND_EMP_ID,Date_chassis_start,date_veh_offln,date_mfg_rlse,PRIM_SHOP_ADDR,SCD_SHOP_ADDR_1,SCD_SHOP_ADDR_2,SCD_SHOP_ADDR_3,SCD_SHOP_ADDR_4
0,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",2,T,UB2964,MHE,on kit cart ...,541,521,05/19/2022,05:02:55,ricrisp,2022-08-18,2022-08-18,,RPK 2,02817R,,,CRITIC
1,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",1,I,NV7601,PLT,INSTALLED BUT NEEDS VERIFIED AND SIGNED OFF.....,207,207,07/25/2022,11:05:44,staffod,2022-08-11,2022-08-12,2022-08-14,TUG03,61113R,,,CRITIC
2,03-40538-002,"FWS-DETROIT,WIF",1,I,NV7659,VPD,TIED TO MOUNTING BRKT.. S WELCH ...,501,501,07/25/2022,08:23:47,MILLETA,2022-08-11,2022-08-12,2022-08-19,60516E,02524R,02524R,,
3,04-16509-010,"PIPE-5""X10""PLN,CRV",1,I,UD5183,MID,production wrote up part and did not nee d the...,249,926,07/28/2022,19:49:52,MASBURY,2022-08-15,2022-08-16,,6600C,02700,,,
4,A17-21151-028,"HOOD-106,BZL",1,Y,UK8929,VPD,VENDOR SHORT ...,,,07/29/2022,10:25:19,,2022-08-18,,,32500,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33911,A12-28407-000,"FT VLV-M2,1V",1,T,NY8456,MHE,on kit cart/ klw ...,926,926,09/13/2022,02:16:40,MELIWIL,2022-09-13,2022-09-13,,QKPULL,60004L,,MIN 1,MAX 1
33912,22-65293-000,TRIM FASCIA-OVHD CONSOLE,1,T,UJ8400,MHE,installed but needs verified and signed off.....,926,926,09/13/2022,06:06:43,jeffgra,2022-09-13,2022-09-13,,RPK 3,61205L,,,
33913,22-65293-000,TRIM FASCIA-OVHD CONSOLE,1,T,UK9324,MHE,in cab per Justin ...,926,926,09/13/2022,06:06:42,jeffgra,2022-09-13,2022-09-13,,RPK 3,61205L,,,
33914,A22-74243-048,"MIR-M2,BRT,HTD,RMT,CUM AAT,LH",1,I,NY8456,MHE,ON KIT CART ...,926,926,09/13/2022,02:12:48,MELIWIL,2022-09-13,2022-09-13,,KTMIRR,MIRSEQ,,,


In [ ]:
#Drop the Duplicates
pou_data.drop_duplicates(subset="Time Found (Pacific)", keep='first')

#Check the row count again to see if the data was cleaned properly
print('Row Count is: ', len(pou_data.axes[0]))

#Timestamp Conversion

##Convert the current Datatype from Object to datatime[ns]

In [ ]:
pou_data['Time Found (Pacific)'] = pd.to_datetime(pou_data['Time Found (Pacific)'])
pou_data.dtypes

0
Item_no                         object
Item Description                object
Qty                             object
Status                          object
Serial Number                   object
Reason Code                     object
Shortage Notes                  object
Team Found                      object
Team Resp                       object
Date Found                      object
Time Found (Pacific)    datetime64[ns]
SHORT_FOUND_EMP_ID              object
Date_chassis_start              object
date_veh_offln                  object
date_mfg_rlse                   object
PRIM_SHOP_ADDR                  object
SCD_SHOP_ADDR_1                 object
SCD_SHOP_ADDR_2                 object
SCD_SHOP_ADDR_3                 object
SCD_SHOP_ADDR_4                 object
dtype: object

##Create temp array for the column then create new column for the Eastern Standard Time Conversion

In [ ]:
list_temp = []
for columns in pou_data['Time Found (Pacific)']:
  list_temp.append(pd.Timestamp(columns, tz = 'US/Pacific').tz_convert('US/Eastern'))
pou_data['time_found_EST'] = list_temp
print(pou_data['time_found_EST'])

0       2022-09-15 08:02:55-04:00
1       2022-09-15 14:05:44-04:00
2       2022-09-15 11:23:47-04:00
3       2022-09-15 22:49:52-04:00
4       2022-09-15 13:25:19-04:00
                   ...           
33911   2022-09-15 05:16:40-04:00
33912   2022-09-15 09:06:43-04:00
33913   2022-09-15 09:06:42-04:00
33914   2022-09-15 05:12:48-04:00
33915   2022-09-15 13:41:42-04:00
Name: time_found_EST, Length: 33916, dtype: datetime64[ns, US/Eastern]


###Set DataTypes for DataFrame

In [ ]:
#Check what datatypes are in the dataframe
pou_data.dtypes

0
Item_no                                     object
Item Description                            object
Qty                                         object
Status                                      object
Serial Number                               object
Reason Code                                 object
Shortage Notes                              object
Team Found                                  object
Team Resp                                   object
Date Found                                  object
Time Found (Pacific)                datetime64[ns]
SHORT_FOUND_EMP_ID                          object
Date_chassis_start                          object
date_veh_offln                              object
date_mfg_rlse                               object
PRIM_SHOP_ADDR                              object
SCD_SHOP_ADDR_1                             object
SCD_SHOP_ADDR_2                             object
SCD_SHOP_ADDR_3                             object
SCD_SHOP_ADDR_4              

In [ ]:
pou_data.dtypes

0
Item_no                                     object
Item Description                            object
Qty                                         object
Status                                      object
Serial Number                               object
Reason Code                                 object
Shortage Notes                              object
Team Found                                  object
Team Resp                                   object
Date Found                                  object
Time Found (Pacific)                datetime64[ns]
SHORT_FOUND_EMP_ID                          object
Date_chassis_start                          object
date_veh_offln                              object
date_mfg_rlse                               object
PRIM_SHOP_ADDR                              object
SCD_SHOP_ADDR_1                             object
SCD_SHOP_ADDR_2                             object
SCD_SHOP_ADDR_3                             object
SCD_SHOP_ADDR_4              

##Create new Dataframe for POU analysis

Data Callouts for the POU Data 

1.   Item_no
2.   Item Description
3.   Serial Number
4.   Reason  
5.   Reason Code
6.   Status
7.   Team Responsible 
8.   Primary Shop Address
9.   C/S Quantity
10.  Qty
11.  Time Found in EST
12.  Date

In [ ]:
#Create the new dataframe to send data into the Google Sheet
pou_data_cleaned = pd.DataFrame(columns=['Item_no', 'Item Description','Serial Number','Reason Code', 'Status','Operating Units', 'Team Responsible', 'Quantity', 'Date', 'Time Found (EST)'])
pou_data_cleaned['Item_no'] = pou_data['Item_no']
pou_data_cleaned['Item Description'] = pou_data['Item Description']
pou_data_cleaned['Reason Code'] = pou_data['Reason Code']
pou_data_cleaned['Serial Number'] = pou_data['Serial Number']
pou_data_cleaned['Status'] = pou_data['Status']
pou_data_cleaned['Team Responsible'] = pd.to_numeric(pou_data['Team Resp'])
pou_data_cleaned['Quantity'] = pou_data['Qty']
pou_data_cleaned['Date'] = pd.to_datetime(pou_data['Date Found'])
pou_data_cleaned['Time Found (EST)'] = pou_data['time_found_EST']
pou_data_cleaned

,Item_no,Item Description,Serial Number,Reason Code,Status,Operating Units,Team Responsible,Quantity,Date,Time Found (EST)
0,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",UB2964,MHE,T,NaN,521.0,2,2022-05-19,2022-09-15 08:02:55-04:00
1,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",NV7601,PLT,I,NaN,207.0,1,2022-07-25,2022-09-15 14:05:44-04:00
2,03-40538-002,"FWS-DETROIT,WIF",NV7659,VPD,I,NaN,501.0,1,2022-07-25,2022-09-15 11:23:47-04:00
3,04-16509-010,"PIPE-5""X10""PLN,CRV",UD5183,MID,I,NaN,926.0,1,2022-07-28,2022-09-15 22:49:52-04:00
4,A17-21151-028,"HOOD-106,BZL",UK8929,VPD,Y,NaN,NaN,1,2022-07-29,2022-09-15 13:25:19-04:00
...,...,...,...,...,...,...,...,...,...,...
33911,A12-28407-000,"FT VLV-M2,1V",NY8456,MHE,T,NaN,926.0,1,2022-09-13,2022-09-15 05:16:40-04:00
33912,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UJ8400,MHE,T,NaN,926.0,1,2022-09-13,2022-09-15 09:06:43-04:00
33913,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UK9324,MHE,T,NaN,926.0,1,2022-09-13,2022-09-15 09:06:42-04:00
33914,A22-74243-048,"MIR-M2,BRT,HTD,RMT,CUM AAT,LH",NY8456,MHE,I,NaN,926.0,1,2022-09-13,2022-09-15 05:12:48-04:00




```
# This is formatted as code
```

#Formatting Data

##Classify the Operating Units

Breakdown of Operating Units - numbers differ from shift to shift

1.   CIW / Western Star: -  100 - 199
2.   Final Cab: 200 - 299
3.   Paint - 300 - 399
4.   Pool 1/Engine Line: 400 - 499
5.   Final Chassis: 500 - 599
6.   Offline - 800 - 899

In [ ]:
#Create new column within the dataframe
pou_data_cleaned['Operating Units'] = 'Workstation'

In [ ]:
def op_units(x):
  if x >= 100 and x < 199:
    return 'CIW/Western'
  elif x >= 200 and x < 299:
    return 'Final Cab'
  elif x >= 300 and x < 399:
    return 'Paint'
  elif x >= 400 and x < 499:
    return 'Pool 1/Engine Line'
  elif x >= 500 and x < 599:
    return 'Final Chassis'
  else:
    return 'Workstation'

In [ ]:
pou_data_cleaned['Operating Units'] = pou_data_cleaned['Team Responsible'].apply(op_units)

In [ ]:
pou_data_cleaned

,Item_no,Item Description,Serial Number,Reason Code,Status,Operating Units,Team Responsible,Quantity,Date,Time Found (EST)
0,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",UB2964,MHE,T,Final Chassis,521.0,2,2022-05-19,2022-09-15 08:02:55-04:00
1,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",NV7601,PLT,I,Final Cab,207.0,1,2022-07-25,2022-09-15 14:05:44-04:00
2,03-40538-002,"FWS-DETROIT,WIF",NV7659,VPD,I,Final Chassis,501.0,1,2022-07-25,2022-09-15 11:23:47-04:00
3,04-16509-010,"PIPE-5""X10""PLN,CRV",UD5183,MID,I,Workstation,926.0,1,2022-07-28,2022-09-15 22:49:52-04:00
4,A17-21151-028,"HOOD-106,BZL",UK8929,VPD,Y,Workstation,NaN,1,2022-07-29,2022-09-15 13:25:19-04:00
...,...,...,...,...,...,...,...,...,...,...
33911,A12-28407-000,"FT VLV-M2,1V",NY8456,MHE,T,Workstation,926.0,1,2022-09-13,2022-09-15 05:16:40-04:00
33912,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UJ8400,MHE,T,Workstation,926.0,1,2022-09-13,2022-09-15 09:06:43-04:00
33913,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UK9324,MHE,T,Workstation,926.0,1,2022-09-13,2022-09-15 09:06:42-04:00
33914,A22-74243-048,"MIR-M2,BRT,HTD,RMT,CUM AAT,LH",NY8456,MHE,I,Workstation,926.0,1,2022-09-13,2022-09-15 05:12:48-04:00


### Format the Timestamp Column

In [ ]:
time_found = pd.Series(pou_data_cleaned['Time Found (EST)'])
new_time_found = time_found.dt.time

In [ ]:
pou_data_cleaned['Time Found'] = new_time_found
del pou_data_cleaned['Time Found (EST)']
pou_data_cleaned

,Item_no,Item Description,Serial Number,Reason Code,Status,Operating Units,Team Responsible,Quantity,Date,Time Found
0,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",UB2964,MHE,T,Final Chassis,521.0,2,2022-05-19,08:02:55
1,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",NV7601,PLT,I,Final Cab,207.0,1,2022-07-25,14:05:44
2,03-40538-002,"FWS-DETROIT,WIF",NV7659,VPD,I,Final Chassis,501.0,1,2022-07-25,11:23:47
3,04-16509-010,"PIPE-5""X10""PLN,CRV",UD5183,MID,I,Workstation,926.0,1,2022-07-28,22:49:52
4,A17-21151-028,"HOOD-106,BZL",UK8929,VPD,Y,Workstation,NaN,1,2022-07-29,13:25:19
...,...,...,...,...,...,...,...,...,...,...
33911,A12-28407-000,"FT VLV-M2,1V",NY8456,MHE,T,Workstation,926.0,1,2022-09-13,05:16:40
33912,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UJ8400,MHE,T,Workstation,926.0,1,2022-09-13,09:06:43
33913,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UK9324,MHE,T,Workstation,926.0,1,2022-09-13,09:06:42
33914,A22-74243-048,"MIR-M2,BRT,HTD,RMT,CUM AAT,LH",NY8456,MHE,I,Workstation,926.0,1,2022-09-13,05:12:48


In [ ]:
pou_data_cleaned

,Item_no,Item Description,Serial Number,Reason Code,Status,Operating Units,Team Responsible,Quantity,Date,Time Found
0,03-33412-875,"ISOLATOR-FTK,BAND,M2,CBOC",UB2964,MHE,T,Final Chassis,521.0,2,2022-05-19,08:02:55
1,A14-16601-000,"STUBSHAFT-STEERING,M2,EPA07",NV7601,PLT,I,Final Cab,207.0,1,2022-07-25,14:05:44
2,03-40538-002,"FWS-DETROIT,WIF",NV7659,VPD,I,Final Chassis,501.0,1,2022-07-25,11:23:47
3,04-16509-010,"PIPE-5""X10""PLN,CRV",UD5183,MID,I,Workstation,926.0,1,2022-07-28,22:49:52
4,A17-21151-028,"HOOD-106,BZL",UK8929,VPD,Y,Workstation,NaN,1,2022-07-29,13:25:19
...,...,...,...,...,...,...,...,...,...,...
33911,A12-28407-000,"FT VLV-M2,1V",NY8456,MHE,T,Workstation,926.0,1,2022-09-13,05:16:40
33912,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UJ8400,MHE,T,Workstation,926.0,1,2022-09-13,09:06:43
33913,22-65293-000,TRIM FASCIA-OVHD CONSOLE,UK9324,MHE,T,Workstation,926.0,1,2022-09-13,09:06:42
33914,A22-74243-048,"MIR-M2,BRT,HTD,RMT,CUM AAT,LH",NY8456,MHE,I,Workstation,926.0,1,2022-09-13,05:12:48


###Send Cleaned data to Google Sheets

In [ ]:
#Convert to type string 
pou_data_cleaned['Time Found'] = pou_data_cleaned['Time Found'].astype(str)
pou_data_cleaned['Date'] = pou_data_cleaned['Date'].astype(str)
pou_data_cleaned['Team Responsible'] = pou_data_cleaned['Team Responsible'].astype(str)

In [ ]:
#Convert dataset into a list
clean_data = pou_data_cleaned.to_numpy().tolist()

#add the headers
headers = pou_data_cleaned.columns.tolist()

#combine with 'clean_data'
dataToWrite = [headers] + clean_data

#Get the Worksheet
wsCleanData = workbook.worksheet('POU_Cleaned_Data')

#Send/Write cleaned data to Google Sheet
wsCleanData.update(None, dataToWrite)

{'spreadsheetId': '1mjE530k7fYKkPh9VPcqfElrrzMkw59uUTPQ34VWH0wA',
 'updatedRange': 'POU_Cleaned_Data!A1:J33917',
 'updatedRows': 33917,
 'updatedColumns': 10,
 'updatedCells': 339170}